In [1]:
import pandas as pd
import sqlalchemy as sa
import os
from dotenv import load_dotenv
load_dotenv()

user = os.environ['REDSHIFT_USER']
password = os.environ['REDSHIFT_PASSWORD']
host = os.environ['REDSHIFT_HOST']
port = os.environ['REDSHIFT_PORT']
db = os.environ['REDSHIFT_DB']


In [2]:
def get_sqlalchemy_engine():
    return sa.create_engine(
        url=f"postgresql://{user}:{password}@{host}:{port}/{db}",
        pool_size=100,
        max_overflow=200,
        client_encoding='utf8',
        executemany_mode='batch',
        echo=False  # True for debbuging
    )


def get_pandas_df_from_query(q):
    global db_conn
    return pd.read_sql_query(q, db_conn)


def ingest_pandas_df(df, schema, table, mode='append'):  # modes 'replace' to recreate table
    global db_conn
    r = df.to_sql(name=table, con=db_conn, schema=schema, if_exists=mode, index=False, chunksize=2000, method='multi')
    print(f'Ingested data for {schema}.{table}, Record Count: {len(df)}, Ingested: {r}')


db_conn = get_sqlalchemy_engine()

In [3]:
query = 'select * from public.shipments limit 10;'
df = get_pandas_df_from_query(query)
len(df)

10

In [4]:
schema = 'zzz_jay_test'
table = 'shipment_test'
ingest_pandas_df(df, schema, table)

Ingested data for zzz_jay_test.shipment_test, Record Count: 10, Ingested: 10
